In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

ptp_1_b_inhibition_classification_path = kagglehub.competition_download('ptp-1-b-inhibition-classification')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import random
import warnings
!pip install plotly

# Visualization Libraries
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline
!pip install plotly

# Machine Learning Libraries
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.ensemble import IsolationForest
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, HistGradientBoostingClassifier

# Set random seed
rs = 42

# Ignore warnings
warnings.filterwarnings("ignore")

# Set color palette for Seaborn
colors= ['#1c76b6', '#a7dae9', '#eb6a20', '#f59d3d', '#677fa0', '#d6e4ed', '#f7e9e5']
sns.set_palette(colors)

## imports

In [ ]:
import pandas as pd
import numpy as np
!pip install rdkit

from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer, f1_score


## data

In [ ]:
df_train = pd.read_csv("/kaggle/input/ptp-1-b-inhibition-classification/train.csv")
df_test = pd.read_csv("/kaggle/input/ptp-1-b-inhibition-classification/test.csv")

In [ ]:
df_train.head()

In [ ]:
# Extract SMILES and Activity Status
smiles_train = df_train['Smiles']
activity_status = df_train['Activity_status'].map({'Active': 1, 'Inactive': 0})

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

## Extracting features

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from rdkit.Chem.rdMolDescriptors import GetMACCSKeysFingerprint

def extract_descriptors(smiles_list, fp_size=2048):
    """
    Extracts molecular descriptors and fingerprints.
    """
    descriptor_names = [
        'MolWt', 'MolLogP', 'NumHDonors', 'NumHAcceptors', 'TPSA', 'FractionCSP3',
        'NumRotatableBonds', 'NumAromaticRings', 'RingCount', 'BalabanJ', 'BertzCT',
        'NumValenceElectrons', 'EState_VSA1', 'Chi0v', 'Chi1v', 'Kappa1', 'Kappa2',
        'LabuteASA', 'HallKierAlpha',
        'EccentricConnectivityIndex',  # New descriptor
        'MolecularComplexity',         # New descriptor
        'ElectrotopologicalStateIndex' # New descriptor
    ]

    calc = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)
    fp_gen = GetMorganGenerator(radius=2, fpSize=fp_size)

    descriptors = []

    for smi in smiles_list:
        try:
            mol = Chem.MolFromSmiles(smi)
            if mol:
                # Compute Molecular Descriptors
                desc_values = list(calc.CalcDescriptors(mol))

                # Compute Morgan Fingerprints
                fp = list(fp_gen.GetFingerprintAsNumPy(mol))

                # Compute MACCS Keys (167-bit)
                maccs_fp = list(GetMACCSKeysFingerprint(mol).ToBitString())
                maccs_fp = [int(bit) for bit in maccs_fp]  # Convert string to binary array

                # Combine Features
                desc_values.extend(fp)
                desc_values.extend(maccs_fp)
            else:
                desc_values = [np.nan] * (len(descriptor_names) + fp_size + 167)
        except Exception:
            desc_values = [np.nan] * (len(descriptor_names) + fp_size + 167)  # Handle errors

        descriptors.append(desc_values)

    # Column Names
    feature_columns = (
        descriptor_names +
        [f'FP_{i}' for i in range(fp_size)] +
        [f'MACCS_{i}' for i in range(167)]
    )

    return pd.DataFrame(descriptors, columns=feature_columns)



In [ ]:
pip install --upgrade rdkit


In [ ]:

# Generate descriptors
X_train = extract_descriptors(smiles_train)
X_test = extract_descriptors(df_test['Smiles'])
y_train = activity_status

# Fill missing descriptor values instead of dropping rows
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)



In [ ]:
# descriptors = []
# for smi in smiles_list:
#     mol = Chem.MolFromSmiles(smi)
#     if mol:
#         desc_values = [
#             Descriptors.MolWt(mol),
#             Descriptors.NumHDonors(mol),
#             Descriptors.NumHAcceptors(mol),
#             Descriptors.MolLogP(mol)
#         ]
#     else:
#         desc_values = [np.nan] * 4
#     descriptors.append(desc_values)

# descriptor_df = pd.DataFrame(descriptors, columns=['MolWt', 'NumHDonors', 'NumHAcceptors', 'MolLogP'])

In [ ]:
X_train

In [ ]:
print(X_train.shape)  # Should be (num_samples, num_descriptors + fp_size)
print(X_test.shape)   # Should match X_train (except num_samples)

## histograms

In [ ]:
# Select only the molecular descriptors (excluding fingerprint bits)
descriptor_columns = X_train.iloc[:, :13]

# Plot histograms
descriptor_columns.hist(figsize=(12, 10), bins=30)
plt.suptitle("Distribution of Molecular Descriptors")
plt.show()

## heatmap

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(descriptor_columns.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap of Molecular Descriptors")
plt.show()


### tried removing outliers using histograms, but gave me worst results

> Ajouter une citation



## outliers

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.01)  # Remove features with variance < 0.01
X_train = pd.DataFrame(selector.fit_transform(X_train))
X_test = pd.DataFrame(selector.transform(X_test))

## norm

In [ ]:
# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


preforming log transformation (and other methods )for skewed data (yes its kindda skewed) didnt do any good -------------
scaling data using smote or udersamling didnt do any good either

## removing highly corrolatted data

In [ ]:
correlation_matrix = X_train.corr().abs()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
high_corr_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.90)]
X_train = X_train.drop(columns=high_corr_features)
X_test = X_test.drop(columns=high_corr_features)

## hyperp using optuna

In [ ]:
import optuna
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, f1_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from catboost import CatBoostClassifier

# Define objective function for Optuna
def objective(trial):
    model_type = trial.suggest_categorical('model_type', ['RandomForest', 'XGBoost', 'LightGBM', 'CatBoost', 'ExtraTrees'])

    if model_type == 'RandomForest':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),  # Reduced range
            'max_depth': trial.suggest_int('max_depth', 3, 15),  # Reduced range
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 5),  # Reduced range
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 3),  # Reduced range
            'n_jobs': -1  # Enable parallel processing
        }
        model = RandomForestClassifier(**params, random_state=42)

    elif model_type == 'XGBoost':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        }
        model = XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss', random_state=42)

    elif model_type == 'LightGBM':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 20, 100),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 30),
        }
        model = LGBMClassifier(**params, random_state=42)

    elif model_type == 'CatBoost':
        params = {
            'iterations': trial.suggest_int('iterations', 100, 300),
            'depth': trial.suggest_int('depth', 3, 8),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10, log=True),
        }
        model = CatBoostClassifier(**params, verbose=0, random_state=42)

    elif model_type == 'ExtraTrees':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 5),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 3),
            'n_jobs': -1  # Enable parallel processing
        }
        model = ExtraTreesClassifier(**params, random_state=42)

    # Perform cross-validation (reduced to 3 folds)
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=kf, scoring=make_scorer(f1_score))

    return np.mean(scores)

# Run optimization (reduced trials)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)  # Reduced from 20 to 10

# Best model & parameters
best_params = study.best_params
print("Best Model:", best_params['model_type'])
print("Best Parameters:", best_params)

# Train the best model
if best_params['model_type'] == 'RandomForest':
    best_model = RandomForestClassifier(**{k: v for k, v in best_params.items() if k != 'model_type'}, random_state=42)
elif best_params['model_type'] == 'XGBoost':
    best_model = XGBClassifier(**{k: v for k, v in best_params.items() if k != 'model_type'}, use_label_encoder=False, eval_metric='logloss', random_state=42)
elif best_params['model_type'] == 'LightGBM':
    best_model = LGBMClassifier(**{k: v for k, v in best_params.items() if k != 'model_type'}, random_state=42)
elif best_params['model_type'] == 'CatBoost':
    best_model = CatBoostClassifier(**{k: v for k, v in best_params.items() if k != 'model_type'}, verbose=0, random_state=42)
elif best_params['model_type'] == 'ExtraTrees':
    best_model = ExtraTreesClassifier(**{k: v for k, v in best_params.items() if k != 'model_type'}, random_state=42)


my anser in genral was all about extracting the right features and fine tuning with optuna

## Old Approach

In [ ]:
# import pandas as pd

# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.neighbors import KNeighborsClassifier
# from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
# from xgboost import XGBClassifier

# # Define models with class weights if needed
# models = {
#     "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced"),
#     "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, scale_pos_weight=10),
#     "LogisticRegression": LogisticRegression(max_iter=1000, class_weight="balanced"),
#     "GradientBoosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
#     "KNN": KNeighborsClassifier(n_neighbors=5),
#     "NaiveBayes": GaussianNB(),
#     "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
#     "ExtraTrees": ExtraTreesClassifier(n_estimators=100, random_state=42, class_weight="balanced"),
#     "LightGBM": LGBMClassifier(n_estimators=100, random_state=42),
#     "CatBoost": CatBoostClassifier(verbose=0, random_state=42)
# }


In [ ]:
# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# best_model = None
# best_score = 0

In [ ]:

# print("\nCross-Validation F1 Scores:")
# for name, model in models.items():
#     scores = cross_val_score(model, df_transformed, y_train, cv=kf, scoring=make_scorer(f1_score))
#     mean_score = np.mean(scores)
#     print(f'{name}: F1 Score = {mean_score:.4f}')

#     if mean_score > best_score:
#         best_score = mean_score
#         best_model = model


In [ ]:
best_score = study.best_value  # This retrieves the best F1 score from optimization
print(best_score)

## the best model the optuna gave me

In [ ]:
# Train best model
print(f'Best Model: {best_model.__class__.__name__} with F1 Score: {best_score:.4f}')
best_model.fit(X_train, y_train)
Best Parameters: {'model_type': 'XGBoost', 'n_estimators': 286, 'max_depth': 4, 'learning_rate': 0.030952931422725427, 'subsample': 0.9942581082777213, 'colsample_bytree': 0.9518891435028527}

## SUBM

In [ ]:
predictions = best_model.predict(X_test)
predicted_labels = ['Active' if pred == 1 else 'Inactive' for pred in predictions]

# ---------------------------- #
# Save Submission
# ---------------------------- #
submission = pd.DataFrame({
    'Molecule ChEMBL ID': df_test['Molecule ChEMBL ID'],
    'Activity_Status': predicted_labels
})

submission.to_csv('10.csv', index=False)
print("\nPredictions saved to submission.csv")